In [1]:
import pandas as pd

# Carregar os dados
df = pd.read_csv('C:/Users/Alice/Downloads/dados_completos.csv', sep=';', encoding='latin-1')

# Remover espaços dos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar os nomes das colunas e as primeiras linhas do DataFrame
print("Colunas do DataFrame:", df.columns)
print(df.head())

# Verificar se a coluna 'Gross' é do tipo string
if df['Gross'].dtype != 'object':
    # Converter para string para aplicar str.replace
    df['Gross'] = df['Gross'].astype(str)

# Converter a coluna 'Gross' para numérica, removendo vírgulas e possíveis caracteres especiais
df['Gross'] = df['Gross'].str.replace(',', '').astype(float)

# Verificar se ainda há valores ausentes nas colunas numéricas e categóricas
print("Valores ausentes por coluna:\n", df.isnull().sum())

# Selecionar as features e o target
features = df.drop(columns=['IMDB_Rating', 'Series_Title'])
target = df['IMDB_Rating']

# Dividir os dados em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Definir colunas numéricas e categóricas
numeric_features = ['Released_Year', 'Runtime', 'No_of_Votes', 'Meta_Score','Gross']
categorical_features = ['Genre_1', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']

# Transformação das features numéricas
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Transformação das features categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Criar o pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicar o pré-processamento nos dados de treino e teste
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Treinar o modelo de regressão linear
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Salvar o modelo em um arquivo joblib
import joblib
joblib.dump(model,'modelo_imdb.pkl')



Colunas do DataFrame: Index(['Unnamed: 0', 'Series_Title', 'Released_Year', 'Certificate', 'Runtime',
       'Genre_1', 'IMDB_Rating', 'Overview', 'Meta_Score', 'Director', 'Star1',
       'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')
   Unnamed: 0                                   Series_Title  Released_Year  \
0           1                                  The Godfather           1972   
1           2                                The Dark Knight           2008   
2           3                         The Godfather: Part II           1974   
3           4                                   12 Angry Men           1957   
4           5  The Lord of the Rings: The Return of the King           2003   

  Certificate  Runtime Genre_1  IMDB_Rating  \
0           A      175   Crime           92   
1          UA      152  Action           90   
2           A      202   Crime           90   
3           U       96   Crime           90   
4           U      201  Act

['modelo_imdb.pkl']

In [3]:
import pandas as pd
import joblib

# Carregar o modelo treinado
with open("modelo_imdb.pkl", 'rb') as file:
    model = joblib.load(file)

# Exemplo de dados para previsão (substitua com seus próprios dados)
dados_para_previsao = pd.DataFrame( {'Released_Year': [1994],
    'Certificate': ['A'],
    'Runtime': [142],
    'Genre_1': ['Drama'],
    'Meta_Score': [80.0],
    'Director': ['Frank Darabont'],
    'Star1': ['Tim Robbins'],
    'Star2': ['Morgan Freeman'],
    'Star3': ['Bob Gunton'],
    'Star4': ['William Sadler'],
    'No_of_Votes': [2343110],  
    'Gross': [28341469] 
})

# Realizar a previsão
X_predicao = preprocessor.transform(dados_para_previsao)
y_pred = model.predict(X_predicao)

# Exibir a previsão
print(f'A previsão de nota do IMDB para o filme é: {y_pred[0]:.2f}')


A previsão de nota do IMDB para o filme é: 95.10
